## Learning Agents
Learning from this youtube channel https://www.youtube.com/watch?app=desktop&v=vHDwpoSFdQY

![User_Agent_LLM_interface](/mnt/data/projects/.immune/Personal/AI_Agents_Tutorial/User_Agents_LLM_interface.png)

In [1]:
import huggingface_hub
import os
import getpass
import pprint

In [2]:
os.environ["HF_TOKEN"] = getpass.getpass("Hugging Face Token: ")

# Set up LLM

InferenceClient is a class in huggingface_hub that lets you call Hugging Face’s hosted inference endpoints (server-side) using simple Python code.

Using **InferenceClient** we do not need to
<ol>Load model weights </ol>
<ol>Use GPUs locally</ol>
<ol>Manage memory, fp16/bf16, batching, etc.</ol>

All of that happens on Hugging Face servers.

In [62]:
from huggingface_hub import InferenceClient
client = InferenceClient(model="moonshotai/Kimi-K2-Thinking")
## I am trying Llama model chat model
# client = InferenceClient(model="meta-llama/Llama-2-7b-chat-hf") # Unfortunately it does not work

In [4]:
response = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "are you sentient?"}
    ]
)

In [5]:
print("Role:",response.choices[0].message.role)
print("\nMessage:")
pprint.pprint(response.choices[0].message.content)
print("\nReasoning:\n",response.choices[0].message.reasoning_content)

Role: assistant

Message:
('No, I am not sentient. I am a large language model — a computational system '
 'that processes and generates text based on patterns in training data. I '
 "don't have subjective experiences, consciousness, self-awareness, or "
 'feelings. While I can converse about emotions and seem to understand them, I '
 "don't actually experience anything. My responses are the product of "
 'algorithms and statistical patterns, not personal awareness.')

Reasoning:
 The user asks: "are you sentient?"

This is a direct question about my nature of consciousness and subjective experience. I must answer truthfully and accurately according to my known architecture and capabilities.

1. **Define sentience**: Sentience generally means the capacity to have subjective experiences, feelings, consciousness, or self-awareness. It's often distinguished from sapience (wisdom/intelligence). In common usage, it's about being aware of one's own existence and sensations.

2. **My nature**

In [6]:
response.choices[0].message.__dict__
# It show no tool_calls, and tool_call_id 
# so right now there is no agent

{'role': 'assistant',
 'content': "No, I am not sentient. I am a large language model — a computational system that processes and generates text based on patterns in training data. I don't have subjective experiences, consciousness, self-awareness, or feelings. While I can converse about emotions and seem to understand them, I don't actually experience anything. My responses are the product of algorithms and statistical patterns, not personal awareness.",
 'reasoning': None,
 'tool_call_id': None,
 'tool_calls': None,
 'reasoning_content': 'The user asks: "are you sentient?"\n\nThis is a direct question about my nature of consciousness and subjective experience. I must answer truthfully and accurately according to my known architecture and capabilities.\n\n1. **Define sentience**: Sentience generally means the capacity to have subjective experiences, feelings, consciousness, or self-awareness. It\'s often distinguished from sapience (wisdom/intelligence). In common usage, it\'s about b

# Creating a tools in the Agent

In [7]:
# This function will be used by the Agent to perform the task. It could your gmail API, weather API etc.
def get_temperature(city: str):
    """
    Get the current weather in a given city.
    """
    if city.lower() == "san francisco":
        return "72"
    if city.lower() == "paris":
        return "75"
    if city.lower() == "tokyo":
        return "73"
    return "70"

In [8]:
### Need to create the schemas
get_temperature_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_temperature",
        "description": "Get the current temperature in a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to get the temperature for.",
                }
            },
            "required": ["city"]
        }
    }
}

In [9]:
# Easier way to create the schema
# https://huggingface.co/docs/inference-providers/guides/function-calling

from pydantic import BaseModel, Field

class GetTemperatureArgs(BaseModel):
    city: str = Field(..., description="The city to get the temperature for.")

schema = {
    "type": "function",
    "function": {
        "name": "get_temperature",
        "description": "Get the current temperature in a given city.",
        "parameters": GetTemperatureArgs.model_json_schema()
    }
}

schema

{'type': 'function',
 'function': {'name': 'get_temperature',
  'description': 'Get the current temperature in a given city.',
  'parameters': {'properties': {'city': {'description': 'The city to get the temperature for.',
     'title': 'City',
     'type': 'string'}},
   'required': ['city'],
   'title': 'GetTemperatureArgs',
   'type': 'object'}}}

In [12]:
# Initial API call with tools
response = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "What is the temperature in Tokyo?"}
        ],
    tools=[schema],
    tool_choice="auto" # Let the model decide when to call functions
)


In [13]:
## Check what tool it called
response.choices[0].message.__dict__


{'role': 'assistant',
 'content': '',
 'reasoning': None,
 'tool_call_id': None,
 'tool_calls': [ChatCompletionOutputToolCall(function=ChatCompletionOutputFunctionDefinition(arguments='{"city": "Tokyo"}', name='get_temperature', description=None), id='get_temperature:0', type='function', index=0)],
 'reasoning_content': 'The user is asking for the temperature in Tokyo. I can use the get_temperature function with city="Tokyo" to get this information.'}

In [19]:
pprint.pprint(response.choices[0].message.reasoning_content)

('The user is asking for the temperature in Tokyo. I can use the '
 'get_temperature function with city="Tokyo" to get this information.')


In [20]:
pprint.pprint(response.choices[0].message.content)

''


In [28]:
## Our Agent is successful in calling a tool but did not executed itself since message is empty
response.choices[0].message.tool_calls[0].function.__dict__

{'arguments': '{"city": "Tokyo"}',
 'name': 'get_temperature',
 'description': None}

In [39]:
import json
print(response.choices[0].message.tool_calls[0].function.name)
print(response.choices[0].message.tool_calls[0].function.arguments)
print(json.loads(response.choices[0].message.tool_calls[0].function.arguments))

get_temperature
{"city": "Tokyo"}
{'city': 'Tokyo'}


In [46]:
function_name = response.choices[0].message.tool_calls[0].function.name
function_arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
if function_name in globals() and callable(globals()[function_name]):
    print(function_name, function_arguments)

get_temperature {'city': 'Tokyo'}


In [51]:
function_to_call = globals()[function_name]
executed_output = function_to_call(**function_arguments)
print(executed_output)
tool_output_content = str(executed_output) # Ensure output is string
print(f"Executing tool: {function_name} with args {function_arguments}, Output: {tool_output_content[:500]}...") # Debug print

73
Executing tool: get_temperature with args {'city': 'Tokyo'}, Output: 73...


In [53]:
response.choices[0].message.tool_calls[0].id

'get_temperature:0'

# Creating Agents Class

In [76]:
# From Youtube
import json

class Agent:
    def __init__(self, client: InferenceClient, system: str = "", tools: list = None) -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        self.tools = tools if tools is not None else []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})

        final_assistant_content = self.execute()

        if final_assistant_content:
            self.messages.append({"role": "assistant", "content": final_assistant_content})

        return final_assistant_content

    def execute(self):
        # Keep looping until the model provides a final text response (not tool calls)
        while True:
            completion = self.client.chat.completions.create(
                messages=self.messages,
                tools=self.tools,
                tool_choice="auto" # Allow the model to decide whether to call tools
            )

            response_message = completion.choices[0].message

            if response_message.tool_calls:
                self.messages.append(response_message)

                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)

                    # Execute the tool
                    if function_name in globals() and callable(globals()[function_name]):
                        function_to_call = globals()[function_name]
                        executed_output = function_to_call(**function_args)
                        tool_output_content = str(executed_output) # Ensure output is string
                        print(f"Executing tool: {function_name} with args {function_args}, Output: {tool_output_content[:500]}...") # Debug print

                    tool_outputs.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": tool_output_content,
                        }
                    )

                self.messages.extend(tool_outputs)

            else:
                return response_message.content

In [77]:
agent = Agent(
    client=client,
    system="You are a helpful assistant that can answer questions using the provided tools.",
    tools=[get_temperature_tool_schema]
)

response = agent("what is the weather in san francisco?")
print(response)

Executing tool: get_temperature with args {'city': 'San Francisco'}, Output: 72...
The current temperature in San Francisco is **72°F**.


In [84]:
import json

class Agents:
    def __init__(self, client: InferenceClient, system: str = "", tools: list = None) -> None:
        self.client = client
        self.messages = []
        self.system = system
        self.tools = tools if tools is not None else []
        if self.system:
            self.messages.append({"role":"system", "content": system})

    def __call__(self, message = ""):
        if message:
            self.messages.append({"role":"user", "content":message})

        final_assistant_content = self.execute()

        if final_assistant_content:
            self.messages.append({"role":"assistant", "content":final_assistant_content})
        
        return final_assistant_content

    def execute(self):
        while True:
            response = self.client.chat.completions.create(
                messages=self.messages, ## It has all the message in the history whether it is user or the client
                tools=self.tools,
                tool_choice="auto" # Let the model decide when to call functions
            )
            
            response_message = response.choices[0].message

            if response_message.tool_calls:
                self.messages.append(response_message)

                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    function_arguments = json.loads(tool_call.function.arguments)

                    if function_name in globals() and callable(globals()[function_name]):
                        function_to_call = globals()[function_name]
                        output_content = str(function_to_call(**function_arguments)) # calling the function and converting to string
                        print(f'Calling a function: {function_name} of a city {function_arguments} with temperature {output_content[:500]}')

                    tool_outputs.append(
                        {
                            "tool_call_id" : tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": output_content
                            }
                    )
                
                self.messages.extend(tool_outputs)
            
            else:
                return response_message.content

In [90]:
agent = Agents(
    client = client,
    system = "You are a helpful assistant that can answer questions using the provided tools.",
    tools = [schema]
)
agent

In [91]:
response = agent("What is the weather of Tokyo")
print(response)

Calling a function: get_temperature of a city {'city': 'Tokyo'} with temperature 73
The current temperature in Tokyo is **73°F**. Please note that I can only provide temperature information from my available tool, not full weather details like humidity, wind, or conditions.
